In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle


In [33]:
## Load the Dataset
data=pd.read_csv("CHurn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [34]:
## Preprocess the Data
## Drop Irrelevant Columns
data=data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [35]:
## Encode Categorical Variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head()

## Gender has only 2 columns(Male, Female), but Geography has 3 columns so we can't use Label Encoding for it.
## If we apply Label Encoding to Geography(France-0, Spain-1, Germany-3), So we should not give these kind of extra importance to 
## Germany or Spain So we Use One Hot Encoding

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [36]:
## Onehot Encoding "Geography"
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [37]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [38]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [39]:
geo_encoder_df=pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [40]:
## Combine One hot encoding columns with the original data
data=pd.concat([data.drop('Geography', axis=1), geo_encoder_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [41]:
## Save the Encoders and scalers

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file) 

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo, file)

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [42]:
## Divide the Dataset into Dependent and Independent features

X=data.drop('Exited', axis=1)
y=data['Exited']

## Split the Data in Training and Testing Sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)

## Scale these Features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [43]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [44]:
X_test

array([[-0.57749609,  0.91324755, -0.6557859 , ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.29729735,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.52560743, -1.09499335,  0.48508334, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.81311987, -1.09499335,  0.77030065, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.41876609,  0.91324755, -0.94100321, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.24540869,  0.91324755,  0.00972116, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [45]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

ANN IMPLEMENTATION

In [46]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense    ## Whenever we need to create a node, it will be using Dense library which is present in layers
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [47]:
print(X_train.shape)
print(X_train.shape[1])

(8000, 12)
12


In [48]:
X_train.shape[0]

8000

In [49]:
(X_train.shape[1],)

(12,)

In [50]:
## Build our ANN Model

model=Sequential([
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)), ## HL1 Connected with input layer
    Dense(32, activation='relu'),                                 ## HL2
    Dense(1, activation='sigmoid')                                 ## Output Layer
])


In [51]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [53]:
## Compile the Model
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])


## For Binary Classification, We use binary cross entropy
## For MultiClass Classification, We use Sparse Categorical cross entropy, Categorical Cross Entropy

In [54]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [55]:
## Set up the Early Stopping
early_stoppping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=5)

In [56]:
## Train the Model

history=model.fit(
    X_train,y_train,validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback,early_stoppping_callback]
)


## y_test is used for comparison wrto accuracy

Epoch 1/100
250/250 [==============================] - 3s 7ms/step - loss: 0.3960 - accuracy: 0.8357 - val_loss: 0.3586 - val_accuracy: 0.8595
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3528 - accuracy: 0.8571 - val_loss: 0.3534 - val_accuracy: 0.8555
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3480 - accuracy: 0.8590 - val_loss: 0.3473 - val_accuracy: 0.8550
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3444 - accuracy: 0.8562 - val_loss: 0.3372 - val_accuracy: 0.8625
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3428 - accuracy: 0.8599 - val_loss: 0.3543 - val_accuracy: 0.8595
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3425 - accuracy: 0.8584 - val_loss: 0.3434 - val_accuracy: 0.8575
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3376 - accuracy: 0.8633 - val_loss: 0.3455 - val_accuracy: 0.8615

In [57]:
model.save('model.h5')

f:\ANN CLASSIFICATION\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [58]:
## Load Tensorboard Extension

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [59]:
%tensorboard --logdir logs/fit20250312-194556

Reusing TensorBoard on port 6007 (pid 7888), started 1 day, 21:56:19 ago. (Use '!kill 7888' to kill it.)